In [1]:
import logging

# Libraries

In [2]:
import pandas as pd
from collections import defaultdict
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Load Crentials

In [4]:
from dotenv import dotenv_values

config = dotenv_values(".env")
BEARER_TOKEN = config['BEARER_TOKEN_2']

# Liked Tweets

In [7]:
import requests
import os
import json

In [8]:
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {BEARER_TOKEN}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

def connect_to_endpoint(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

def main(url, params):
    json_response = connect_to_endpoint(url, params)
    # print(json.dumps(json_response, indent=4, sort_keys=True))
    return json_response

In [9]:
# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields
query_params = {
    'user.fields': 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld',
    'tweet.fields': 'created_at,public_metrics,entities,lang,possibly_sensitive,reply_settings,source,in_reply_to_user_id,geo',
    'expansions': 'author_id',
}
search_url = "https://api.twitter.com/2/users/919185985132466176/liked_tweets"

output_tweets = main(search_url, query_params)

In [10]:
output_tweets

{'data': [{'text': 'Vefa borcu adına imkanı olanlar için...\nYavrumuzun cenazesi Yarın ikindide Sincan Çimşit mezarlığında kalkacakmış\nNahit Emre Güney\n\n#BuSansürHepimize https://t.co/S300MCLYot',
   'possibly_sensitive': False,
   'lang': 'tr',
   'source': 'Twitter for Android',
   'author_id': '1402306656046915585',
   'entities': {'urls': [{'start': 150,
      'end': 173,
      'url': 'https://t.co/S300MCLYot',
      'expanded_url': 'https://twitter.com/secret_salt_/status/1580631216373395456/video/1',
      'display_url': 'pic.twitter.com/S300MCLYot',
      'media_key': '7_1580631196870008834'}],
    'hashtags': [{'start': 132, 'end': 149, 'tag': 'BuSansürHepimize'}]},
   'created_at': '2022-10-13T18:47:03.000Z',
   'reply_settings': 'everyone',
   'public_metrics': {'retweet_count': 61,
    'reply_count': 2,
    'like_count': 67,
    'quote_count': 0},
   'id': '1580631216373395456',
   'edit_history_tweet_ids': ['1580631216373395456']},
  {'text': 'KHK uygulamaları, \n21. yy 